In [1]:
import sys
sys.path.append('../')

In [2]:
import warnings
import logging
import time
import psutil
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
from multiprocessing import Manager
from multiprocessing.managers import BaseManager


from its_logging.logger_config import logger
from utils.its_utils import clip_to_california, get_wfr_tf_template
from utils.gdf_utils import repair_geometries, show_columns, verify_gdf_columns
from utils.add_common_columns import add_common_columns
from utils.enrich_polygons import enrich_polygons
from utils.keep_fields import keep_fields
from utils.assign_domains import assign_domains
from utils.save_gdf_to_gdb import save_gdf_to_gdb

In [5]:
a_reference_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
start_year, end_year = 2020, 2025

process = psutil.Process(os.getpid())

In [4]:
from multiprocessing.managers import NamespaceProxy, BaseManager
from pandas import DataFrame
import inspect
import os

class MyDataFrame(DataFrame):
    def __getstate__(self):
        print(f'dataframe being pickled in pid {os.getpid()}')
        return super().__getstate__()

    def __setstate__(self, state):
        print(f'dataframe being unpickled in pid {os.getpid()}')
        print()
        return super().__setstate__(state)


class ObjProxy(NamespaceProxy):
    """Returns a proxy instance for any user defined data-type. The proxy instance will have the namespace and
    functions of the data-type (except private/protected callables/attributes). Furthermore, the proxy will be
    pickable and can its state can be shared among different processes. """

    @classmethod
    def populate_obj_attributes(cls, real_cls):
        DISALLOWED = set(dir(cls))
        ALLOWED = ['__sizeof__', '__eq__', '__ne__', '__le__', '__repr__', '__dict__', '__lt__',
                   '__gt__']
        DISALLOWED.add('__class__')
        new_dict = {}
        for (attr, value) in inspect.getmembers(real_cls, callable):
            if attr not in DISALLOWED or attr in ALLOWED:
                new_dict[attr] = proxy_wrap(attr)
        return new_dict


def proxy_wrap(attr):
    """ This method creates function that calls the proxified object's method."""
    def f(self, *args, **kwargs):

        # _callmethod is the method that proxies provided by multiprocessing use to call methods in the proxified object
        return self._callmethod(attr, args, kwargs)

    return f

# Create a class during runtime
new_dict = ObjProxy.populate_obj_attributes(MyDataFrame)
DataFrameObjProxy = type("DataFrameObjProxy", (ObjProxy,), new_dict)
BaseManager.register('DataFrame', MyDataFrame, DataFrameObjProxy, exposed=tuple(dir(DataFrameObjProxy)))

In [ ]:
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\USFS_{}_{}.gdb".format(start_year, end_year)
if __name__ == '__main__':
    from enrich.enrich_USFS import enrich_USFS
    #from utils.mp_df import MyDataFrame, ObjProxy
    
    

    
    
    
    
    region_ids = ["05"]#["04", "05", "06"]
    for region_id in region_ids:    
        usfs_input_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\USFS_FACTS_2023\USFS_FACTS_2023_20240620_uploadEmilyBrodie\Actv_CommonAttribute_PL_Region{}.gdb'.format(region_id)
        usfs_input_layer_name = "Actv_CommonAttribute_PL"
        output_layer_name = f"USFS_Region{region_id}_enriched_{datetime.today().strftime('%Y%m%d')}"
        # init multiprocessing manager in main module for Windows fork
        manager = BaseManager()
        manager.start()
        
        enrich_USFS(usfs_input_gdb_path,
                    usfs_input_layer_name,
                    a_reference_gdb_path,
                    start_year,
                    end_year,
                    output_gdb_path,
                    output_layer_name,
                    manager=manager)


In [4]:
from enrich.enrich_Timber_Nonspatial import enrich_Timber_Nonspatial

In [9]:


tn_input_excel_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\Industry_nonspatial_2023\Timber_Industry_Acres_{}.xlsx'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\Timber_Nonspatial_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"Timber_Nonspatial_{datetime.today().strftime('%Y%m%d')}"



In [4]:
from enrich.enrich_NPS import enrich_NPS_from_gdb

In [5]:
nps_gdb_path = r'D:\WORK\wildfire\Interagency-Tracking-System\2023\NPS_2023\New_NPS_2023_20240625_ReisThomasViaUpload_1.gdb'
nps_layer_name = 'NPS_2023_20240625_ReisThomasViaUpload2'
output_gdb_path =  r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\NPS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"NPS_enriched_{datetime.today().strftime('%Y%m%d')}"



In [3]:
from enrich.enrich_BLM import enrich_BLM

In [6]:
blm_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\BLM_2023\BLM_2010_2023_fromReisThomasViaUpload.gdb"
blm_input_layer_name = "BLM_2010_2023_fromReisThomasViaUpload"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"BLM_enriched_{datetime.today().strftime('%Y%m%d')}"

In [7]:

enrich_BLM(blm_input_gdb_path,
               blm_input_layer_name,
               a_reference_gdb_path,
               start_year,
               end_year,
               output_gdb_path,
               output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-04-25 09:25:39,944 INFO  [enrich.enrich_BLM     ]  Load the BLM data into a GeoDataFrame
2025-04-25 09:25:40,047 INFO  [enrich.enrich_BLM     ]     time for loading BLM_2010_2023_fromReisThomasViaUpload: 0.10314106941223145
2025-04-25 09:25:40,048 INFO  [enrich.enrich_BLM     ]     all required columns are present.
2025-04-25 09:25:40,049 INFO  [enrich.enrich_BLM     ]  Performing Standardization...
2025-04-25 09:25:40,049 INFO  [enrich.enrich_BLM     ]     step 1/15 Clip Features to California...
2025-04-25 09:25:52,703 INFO  [enrich.enrich_BLM     ]     step 2/15 Repairing Geometry...
2025-04-25 09:25:53,173 INFO  [enrich.enrich_BLM     ]     step 3/15 Adding Common Columns...
2025-04-25 09:25:53,199 INFO  [enrich.enrich_BLM     ]     step 4/15 Transfering Values...
2025-04-25 09:25:53,201 INFO  [enrich.enrich_BLM     ]     step 5/15 Calculating Start and End Date...
2025-04-25 09:25:53,237 INFO  [enrich.enrich_BLM     ]     step 6/15 Calculating Status...
2025-04-25 09:25:53,23

2025-04-25 09:29:39,108 INFO  [utils.crosswalk       ]              cross step 5/8 calculating objective...
2025-04-25 09:29:39,109 INFO  [utils.crosswalk       ]              cross step 6/8 calculate category
2025-04-25 09:29:39,113 INFO  [utils.crosswalk       ]              cross step 7/8 standardize domains
2025-04-25 09:29:39,124 INFO  [utils.crosswalk       ]              cross step 8/8 counts towards MAS
2025-04-25 09:29:39,124 INFO  [utils.counts_to_mas   ]           Calculating Counts to MAS
2025-04-25 09:29:39,125 INFO  [utils.counts_to_mas   ]              counts step 1/8: set to 'NO'
2025-04-25 09:29:39,125 INFO  [utils.counts_to_mas   ]              counts step 2/8: select by bounding years (2020-2025)
2025-04-25 09:29:39,125 INFO  [utils.counts_to_mas   ]              counts step 3/8: set to 'YES' if activity description is in the list
2025-04-25 09:29:39,127 INFO  [utils.counts_to_mas   ]              counts step 4/8: set to 'NO' if not 'Acres'
2025-04-25 09:29:39,127 IN

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\BLM_2020_2025.gdb


In [8]:
from enrich.enrich_CNRA import enrich_CNRA

In [9]:

cnra_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CNRA_2023\CNRA_Tracker_Data_Export_20241120.gdb"
cnra_polygon_layer_name = "TREATMENT_POLY"
cnra_line_layer_name = "TREATMENT_LINE"
cnra_point_layer_name = "TREATMENT_POINT"
cnra_project_polygon_layer_name = "PROJECT_POLY"
cnra_activity_layer_name = "ACTIVITIES"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CNRA_enriched_{datetime.today().strftime('%Y%m%d')}"


In [10]:

enrich_CNRA(cnra_input_gdb_path,
            cnra_polygon_layer_name,
            cnra_line_layer_name,
            cnra_point_layer_name,
            cnra_project_polygon_layer_name,
            cnra_activity_layer_name,
            a_reference_gdb_path,
            start_year,
            end_year,
            output_gdb_path,
            output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")


2025-02-19 11:17:11,219 INFO  [enrich.enrich_CNRA    ]  Load the CNRA polygon layer into a GeoDataFrame
2025-02-19 11:17:11,691 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-19 11:17:12,390 INFO  [enrich.enrich_CNRA    ]  Load the CNRA line layer into a GeoDataFrame
2025-02-19 11:17:12,405 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-19 11:17:12,409 INFO  [enrich.enrich_CNRA    ]  Load the CNRA point layer into a GeoDataFrame
2025-02-19 11:17:12,427 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-19 11:17:12,428 INFO  [enrich.enrich_CNRA    ]  Load the CNRA project polygon layer into a GeoDataFrame
2025-02-19 11:17:12,627 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-19 11:17:13,060 INFO  [enrich.enrich_CNRA    ]  Load the CNRA activity layer into a DataFrame
2025-02-19 11:17:13,727 INFO  [enrich.enrich_CNRA    ]     all required columns are present.
2025-02-19 11:

2025-02-19 11:20:57,327 INFO  [utils.crosswalk       ]           Calculating Crosswalking Activites...
2025-02-19 11:20:57,327 INFO  [utils.crosswalk       ]              Load Crosswalk table...
2025-02-19 11:20:57,379 INFO  [utils.crosswalk       ]              cross step 1/8 add join
2025-02-19 11:20:57,429 INFO  [utils.crosswalk       ]              cross step 2/8 calculate activities
2025-02-19 11:20:57,432 INFO  [utils.crosswalk       ]              cross step 3/8 calculate residue fate field
2025-02-19 11:20:57,434 INFO  [utils.crosswalk       ]              cross step 4/8 select attribute by layer
2025-02-19 11:20:57,437 INFO  [utils.crosswalk       ]              cross step 5/8 calculating objective...
2025-02-19 11:20:57,470 INFO  [utils.crosswalk       ]              cross step 6/8 calculate category
2025-02-19 11:20:57,756 INFO  [utils.crosswalk       ]              cross step 7/8 standardize domains
2025-02-19 11:20:58,592 INFO  [utils.crosswalk       ]              cross s

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb


2025-02-19 11:21:07,850 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb CNRA_enriched_20250219_polygon
2025-02-19 11:21:07,886 INFO  [enrich.enrich_CNRA    ]  Enrich the CNRA lines...
2025-02-19 11:21:07,887 INFO  [enrich.enrich_CNRA    ]     Part 1 Prepare Features
2025-02-19 11:21:07,889 INFO  [enrich.enrich_CNRA    ]     Part 2 Prepare Activity Table
2025-02-19 11:21:08,432 INFO  [enrich.enrich_CNRA    ]        step 2/17 remove milliseconds from dates
2025-02-19 11:21:08,448 INFO  [enrich.enrich_CNRA    ]        step 3/17 create standardized activity table
2025-02-19 11:21:08,453 INFO  [enrich.enrich_CNRA    ]        step 4/17 import activities into standardized table
2025-02-19 11:21:08,481 INFO  [enrich.enrich_CNRA    ]        step 5/17 calculate unique Treatment ID with postfix '-CNRA'
2025-02-19 11:21:08,493 INFO  [enrich.enrich_CNRA    ]     Part 3 - Combine CNRA Features and Activity Tabl

2025-02-19 11:21:22,175 INFO  [utils.assign_domains  ]        Created domain 'D_CNTY' with 59 values
2025-02-19 11:21:22,177 INFO  [utils.assign_domains  ]        Created domain 'D_IN_WUI' with 4 values
2025-02-19 11:21:22,180 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVDSCRP' with 61 values
2025-02-19 11:21:22,182 INFO  [utils.assign_domains  ]        Created domain 'D_ACTVCAT' with 8 values
2025-02-19 11:21:22,184 INFO  [utils.assign_domains  ]        Created domain 'D_USERDEFINED' with 2 values
2025-02-19 11:21:22,186 INFO  [utils.assign_domains  ]        Created domain 'D_BVT' with 8 values
2025-02-19 11:21:22,188 INFO  [utils.assign_domains  ]        Created domain 'D_RESIDUEFATE' with 15 values
2025-02-19 11:21:22,190 INFO  [utils.assign_domains  ]        Created domain 'D_UOM' with 6 values
2025-02-19 11:21:22,192 INFO  [utils.assign_domains  ]        Created domain 'D_TASKFORCE' with 5 values
2025-02-19 11:21:22,193 INFO  [utils.assign_domains  ]        Created 

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb


2025-02-19 11:21:22,910 INFO  [enrich.enrich_CNRA    ]        step 2/17 remove milliseconds from dates
2025-02-19 11:21:22,929 INFO  [enrich.enrich_CNRA    ]        step 3/17 create standardized activity table
2025-02-19 11:21:22,934 INFO  [enrich.enrich_CNRA    ]        step 4/17 import activities into standardized table
2025-02-19 11:21:22,964 INFO  [enrich.enrich_CNRA    ]        step 5/17 calculate unique Treatment ID with postfix '-CNRA'
2025-02-19 11:21:22,984 INFO  [enrich.enrich_CNRA    ]     Part 3 - Combine CNRA Features and Activity Table
2025-02-19 11:21:22,984 INFO  [enrich.enrich_CNRA    ]        step 6/17 join polygon table and activity table
2025-02-19 11:21:23,009 INFO  [enrich.enrich_CNRA    ]     Part 4 Prepare Project Table
2025-02-19 11:21:23,011 INFO  [enrich.enrich_CNRA    ]        step 7/17 calculate unique Project ID if null
2025-02-19 11:21:23,032 INFO  [enrich.enrich_CNRA    ]     Part 5 Join Project Table to Features/Activities
2025-02-19 11:21:23,043 INFO  

2025-02-19 11:22:09,631 INFO  [utils.assign_domains  ]        Created domain 'D_FNDSRC' with 11 values
2025-02-19 11:22:09,634 INFO  [utils.assign_domains  ]        Created domain 'D_AGENCY' with 8 values
2025-02-19 11:22:09,638 INFO  [utils.assign_domains  ]        Created domain 'D_ORGANIZATION' with 31 values
2025-02-19 11:22:09,640 INFO  [utils.assign_domains  ]        Warning: 1 rows with NULL values were dropped from 'D_DATASTATUS'
2025-02-19 11:22:09,641 INFO  [utils.assign_domains  ]        Created domain 'D_DATASTATUS' with 2 values
2025-02-19 11:22:09,644 INFO  [utils.assign_domains  ]        Created domain 'D_DATAMSG' with 3 values
2025-02-19 11:22:09,647 INFO  [utils.assign_domains  ]        Created domain 'D_VERFIEDMSG' with 2 values
2025-02-19 11:22:09,649 INFO  [utils.assign_domains  ]        Created domain 'D_TRMT_GEOM' with 4 values
2025-02-19 11:22:09,650 INFO  [utils.assign_domains  ]        Assign domains to project-related columns
2025-02-19 11:22:09,657 INFO  [uti

D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb


2025-02-19 11:22:10,181 INFO  [its_logging.logger_config]  Memory usage: 5211.73 MB


In [11]:
# temp file path
caltrans_input_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\2023\CALTRANS_2023\Caltrans_Vegetation_Management_20_23.gdb"
tree_activity_layer_name = "Caltrans_Vegetation_Management_Trees_ActivitiesTable_20_23"
tree_treatment_layer_name = "Caltrans_Vegetation_Management_Trees_Treatments_20_23"
road_activity_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_ActivitiesTable_20_23"
road_treatment_layer_name = "Caltrans_Vegetation_Management_RoadsideLandscape_Treatments_20_23"
start_year = 2021
end_year = 2023
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CalTRANS_{}_{}.gdb".format(start_year, end_year)
output_layer_name = f"CalTRANS_enriched_{datetime.today().strftime('%Y%m%d')}"